In [1]:
import numpy as  np
import pandas as pd

In [2]:
data = pd.read_csv('seeds_dataset.txt', delimiter='\t')

In [3]:
data.head()


,15.26,14.84,0.871,5.763,3.312,2.221,5.22,1
0,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
1,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
2,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
3,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
4,14.38,14.21,0.8951,5.386,3.312,2.462,4.956,1


In [4]:
data = np.array(data)

In [5]:
m, n = data.shape
np.random.shuffle(data)

data_test = data[0:40].T
Y_test = data_test[-1]
X_test = data_test[:-1]

data_train = data[40:m].T
Y_train = data_train[-1]
X_train = data_train[:-1]



In [6]:
X_train[:, 0].shape

(7,)

In [7]:
Y_test

array([3., 2., 2., 3., 2., 1., 3., 1., 3., 2., 2., 1., 3., 2., 1., 3., 2.,
       2., 2., 3., 2., 2., 3., 3., 3., 1., 3., 2., 1., 3., 3., 3., 3., 3.,
       1., 3., 2., 2., 1., 1.])

In [8]:
def init_params():
    W1 = np.random.rand(5, 7) - 0.5
    b1 = np.random.rand(5, 1) - 0.5
    W2 = np.random.rand(3, 5) - 0.5
    b2 = np.random.rand(3, 1) - 0.5
    return W1, b1, W2, b2

In [9]:
def ReLU(Z):
    return np.maximum(Z, 0)


def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0
    
def one_hot(Y):
    num_classes = int(Y.max())
    one_hot_Y = np.zeros((Y.size, num_classes))
    one_hot_Y[np.arange(Y.size), Y.astype(int) - 1] = 1
    return one_hot_Y


In [10]:
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y.T
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1_new = W1 - alpha * dW1
    b1_new = b1 - alpha * db1    
    W2_new = W2 - alpha * dW2  
    b2_new = b2 - alpha * db2    
    return W1_new, b1_new, W2_new, b2_new

In [59]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    #print(predictions + 1, Y)
    return np.sum(predictions + 1 == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if (i % 10 == 0 and i < 100) or (i > 450 and i % 10  ==  0):
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [76]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.01, 500)

Iteration:  0
0.3136094674556213
Iteration:  10
0.3905325443786982
Iteration:  20
0.5207100591715976
Iteration:  30
0.5621301775147929
Iteration:  40
0.5798816568047337
Iteration:  50
0.6272189349112426
Iteration:  60
0.6272189349112426
Iteration:  70
0.6331360946745562
Iteration:  80
0.6390532544378699
Iteration:  90
0.6390532544378699
Iteration:  460
0.893491124260355
Iteration:  470
0.893491124260355
Iteration:  480
0.893491124260355
Iteration:  490
0.893491124260355


In [77]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions + 1

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

In [78]:
predictions = test_prediction(28, W1, b1, W2, b2)

Prediction:  [3]
Label:  3.0


In [79]:
def test_predictions(X_test, W1, b1, W2, b2):
    predictions = make_predictions(X_test, W1, b1, W2, b2)
    return predictions

In [80]:
def calculate_accuracy(predictions, Y_test):
    return np.sum(predictions == Y_test) / len(Y_test)

In [81]:
predictions = test_predictions(X_test, W1, b1, W2, b2)

In [83]:
accuracy = calculate_accuracy(predictions, Y_test)
print("Accuracy: {:.2%}".format(accuracy))

Accuracy: 90.00%
